In [3]:
import pykokkos as pk
import numpy as np


@pk.workunit()
# d_x_ar,
# d_v_ar,
# d_E_ar,
# d_x_ar=pk.ViewTypeInfo(space=pk.HostSpace),
# d_v_ar=pk.ViewTypeInfo(space=pk.HostSpace),
# d_E_ar=pk.ViewTypeInfo(space=pk.HostSpace),
# )
def pk_advection_kernel(
    tid: int,
    Nc: int,
    d_x_ar: pk.View1D[float],
    d_v_ar: pk.View1D[float],
    d_E_ar: pk.View1D[float],
    stride: int,
):
    # Looping and doing all the particles
    for i in range(tid, Nc, stride):
        if i == 0:
            printf("hi from kernel \n")
        d_x: float = d_x_ar[i]
        d_v: float = d_v_ar[i]
        d_E: float = d_E_ar[i]

        # Advection in v-space
        d_v += d_E

        # Advection in x-space
        d_x += d_v

        # Reflective boundary condition
        if d_x > 1:
            d_x = 1 - (d_x - 1)
            d_v = -d_v
        elif d_x < 0:
            d_x = -d_x
            d_v = -d_v

        # Put data back into arrays
        d_x_ar[i] = d_x
        d_v_ar[i] = d_v
        d_E_ar[i] = d_E


def advect(Nc, d_x_ar, d_v_ar, d_E_ar, threads_per_block, num_blocks, i):
    num_threads: int = num_blocks * threads_per_block
    print("Launch Kernel", i, flush=True)
    # Launch PyKokkos kernel
    pk.parallel_for(
        num_threads,
        pk_advection_kernel,
        Nc=Nc,
        d_x_ar=d_x_ar,
        d_v_ar=d_v_ar,
        d_E_ar=d_E_ar,
        stride=num_threads,
    )
    print("Finish Kernel", i, flush=True)

threads_per_block = 64
num_blocks = 1024

N = 100000
num_steps = 10

x_ar = np.random.rand(N)
v_ar = 0.01 * np.random.rand(N)
E_ar = np.zeros(N)

d_x_ar = pk.array(x_ar)
d_v_ar = pk.array(v_ar)
d_E_ar = pk.array(E_ar)

print("\n Launching main Parla task to run for", num_steps, "steps\n")

advect(N, d_x_ar, d_v_ar, d_E_ar, threads_per_block, num_blocks, 1)


 Launching main Parla task to run for 10 steps

Launch Kernel 1


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ipykernel_528/2858469234.py'